In [11]:
import json
import csv
import geocoder
from pprint import pprint


with open(".passwd/geocoder", "r") as f:
    token = f.read()

with open("./geoip/geoip.json", "r") as f:
    geoip = f.read()
    geoip = json.loads(geoip)    

pop_list = []
city_list = []
for country in geoip:
    for state in geoip[country]:
        for city in geoip[country][state]:
            city_list.append(city)
            for pop in geoip[country][state][city]["ips"]:
                if pop[1] != "undefined.hostname.localhost.":
                    location_code = pop[1].split('.')[1]
                    pop_list.append(location_code)

pop_list = sorted(list(set(pop_list)))
city_list = sorted(list(set(city_list)))

with open("./geoip/pop.csv", newline='') as csvfile:
    reader = csv.reader(csvfile, delimiter=',')
    
    with open("./geoip/pop_location.csv", 'w', newline='') as output:
        writer = csv.writer(output, delimiter=',')
        for row in reader:
            print(row[1])
            location = geocoder.bing("{},{}".format(row[1], row[2]), key=token)
            location = location.json
            writer.writerow([row[0], row[1], location["lat"], location["lng"]])


Auckland
Atlanta
Bogota
Chicago
Dallas
Denver
Dutch Harbor
Frankfurt
Fortaleza
Iqaluit
Jakarta
Jakarta
Karachi
Lagos
Lima
London
Los Angeles
Madrid
Mumbai
Manila
New York
Perth
Point Hope
Mexico City
Redmond
Singapore
Santiago
Sao Paulo
Seattle
Sydney
Tokyo


In [12]:
pop_json = {
    "type": "FeatureCollection",
    "features": []
}


def get_city_subnet(city: str):
  description = ""
  for country in geoip:
    for state in geoip[country]:
        if city in geoip[country][state].keys():
          for ip in geoip[country][state][city]["ips"]:
              description += "{}<br>{}<br>".format(ip[0], ip[1])
          return description


with open("./geoip/pop_location.csv", newline='') as csvfile:
    reader = csv.reader(csvfile, delimiter=',')
    for row in reader:
        pprint(row[1])
        if row[1] in city_list:
          description = row[0] + "<br>" + get_city_subnet(row[1])
        else:
          description = row[0]
      
      
        pop_json["features"].append({
            "type": "Feature",
            "geometry": {
              "type": "Point",
              "coordinates": [float(row[3]), float(row[2])]
            },
            "properties": {
              "title": "{} PoP".format(row[1]),
              "description": description,
            }
          })

json.dump(pop_json, open("./geoip/pop.json", "w"), indent=4)

'Auckland'
'Atlanta'
'Bogota'
'Chicago'
'Dallas'
'Denver'
'Dutch Harbor'
'Frankfurt'
'Fortaleza'
'Iqaluit'
'Jakarta'
'Jakarta'
'Karachi'
'Lagos'
'Lima'
'London'
'Los Angeles'
'Madrid'
'Mumbai'
'Manila'
'New York'
'Perth'
'Point Hope'
'Mexico City'
'Redmond'
'Singapore'
'Santiago'
'Sao Paulo'
'Seattle'
'Sydney'
'Tokyo'


In [13]:
import json
import sys
import time
import math
import geocoder
from pprint import pprint

pop_list = {}
with open("./geoip/pop_location.csv", newline='') as csvfile:
    reader = csv.reader(csvfile, delimiter=',')
    for row in reader:
        pop_list[row[0]] = {
            "name": row[1],
            "lat": row[2],
            "lon": row[3]
        }
        
with open("./geoip/geoip.json", "r") as f:
    geoip = f.read()
    geoip = json.loads(geoip)    

links = {
    "type": "FeatureCollection",
    "features": []
}

count = 0

for country in geoip:
    for state in geoip[country]:
        for city in geoip[country][state]:
            count += 1
            target_pops = []
            for ip in geoip[country][state][city]["ips"]:
                if ip[1] != "undefined.hostname.localhost.":
                    code = ip[1].split('.')[1]
                    target_pops.append(code)
            target_pops = list(set(target_pops))
            print(target_pops)
            
            g = geocoder.bing("{}, {}, {}".format(city, state, country), key=token)
            if g.json is None:    
                g = geocoder.bing("{}, {}".format(city, country), key=token)
                if g.json is None:
                    g = geocoder.bing("{}".format(city), key=token)
                    
            source_gps = g.json
            if source_gps is None:
                print("{}, {}".format(city, country))

            for pop in target_pops:
                if pop in ["rdmdwax3", "rdmdwax3rk3"]:
                    pop = "rdmdwax3/rdmdwax3rk3"      
                elif pop in ["sttlwax1", "sttlwax9"]:
                    pop = "sttlwax1/sttlwax9"            
                target_lon = float(pop_list[pop]["lon"])
                source_lon = source_gps["lng"]
                
                if math.fabs(target_lon - source_lon) > 180:
                    if target_lon > 0:
                        target_lon -= 360
                    else:
                        target_lon += 360
                
                feature = {
                    "type": "Feature",
                    "properties": {
                            "title": "{}->{}".format(city, pop_list[pop]["name"])
                        },
                    "geometry": {
                        "type": "LineString",
                        "coordinates": [
                            [
                                source_lon,
                                source_gps["lat"]
                            ],
                            [
                                target_lon,
                                float(pop_list[pop]["lat"])
                            ]
                        ]
                    }
                }
                links["features"].append(feature)

json.dump(links, open("./geoip/links.json", "w"), indent=4)

['tkyojpn1', 'mnlaphl1']
['mmbiind1']
['krcipak1']
['jtnaidn1', 'sydyaus1', 'acklnzl1']
['prthaus1']
['sngesgp1', 'jtnaidn2', 'jtnaidn1', 'sydyaus1']
['tkyojpn1', 'sydyaus1']
['sngesgp1', 'mnlaphl1']
['sngesgp1', 'tkyojpn1']
['qrtomex1', 'lsancax1', 'dllstxx1']
['chcoilx1']
['nwyynyx1']
['rdmdwax3', 'frntdeu1', 'sttlwax9', 'lndngbr1', 'rdmdwax3rk3', 'sttlwax1']
['lsancax1']
['dllstxx1']
['dnvrcox1']
['atlagax1']
['rdmdwax3']
['dtcrakx1']
['pthpakx']
['pthpakx']
['sttlwax1']
['chcoilx1', 'sttlwax1']
['chcoilx1', 'rdmdwax3']
['chcoilx1', 'rdmdwax3']
['nwyynyx1', 'chcoilx1', 'rdmdwax3']
['nwyynyx1']
['nwyynyx1']
['nwyynyx1']
['iqltcan1', 'pthpakx']
['mdrdesp1', 'lgosnga1']
['sydyaus1', 'acklnzl1']
['limaper1', 'sntochl1']
['limaper1', 'bgtacol1']
['limaper1', 'bgtacol1']
['frntdeu1', 'mdrdesp1', 'lndngbr1', 'lgosnga1']
['bgtacol1', 'limaper1']
['frntdeu1', 'lndngbr1']
['frntdeu1', 'mdrdesp1', 'lndngbr1']
['frntdeu1', 'mdrdesp1', 'lndngbr1']
['frntdeu1', 'mdrdesp1', 'lndngbr1']
['frntdeu1'

In [16]:
import json
import sys
import time
import geocoder
from pprint import pprint

pop_list = {}
with open("./geoip/pop_location.csv", newline='') as csvfile:
    reader = csv.reader(csvfile, delimiter=',')
    for row in reader:
        pop_list[row[0]] = {
            "name": row[1],
            "lat": row[2],
            "lon": row[3]
        }
        
with open("./geoip/geoip.json", "r") as f:
    geoip = f.read()
    geoip = json.loads(geoip)    

city_json = {
    "type": "FeatureCollection",
    "features": []
}


for country in geoip:
    for state in geoip[country]:
        for city in geoip[country][state]:
            print(city)
            g = geocoder.bing("{}, {}, {}".format(city, state, country), key=token)
            if g.json is None:    
                g = geocoder.bing("{}, {}".format(city, country), key=token)
                if g.json is None:
                    g = geocoder.bing("{}".format(city), key=token)
            source_gps = g.json

            description = ""
            for ip in geoip[country][state][city]["ips"]:
                description += "{}<br>{}<br>".format(ip[0], ip[1])
            
            city_json["features"].append({
                "type": "Feature",
                "geometry": {
                "type": "Point",
                "coordinates": [float(source_gps["lng"]), float(source_gps["lat"])]
                    },
                "properties": {
                    "title": city,
                    "description": description,
                }
            })

json.dump(city_json, open("./geoip/city.json", "w"), indent=4)

Manila
Mumbai
Karachi
Sydney
Perth
Jakarta
Tokyo
Kuala Lumpur
Singapore
Mexico City
Chicago
New York
Seattle
Los Angeles
Dallas
Denver
Atlanta
Minneapolis
Dutch Harbor
Nome
Miami
Vancouver
Calgary
Winnipeg
Regina
Toronto
Montreal
Fredericton
Halifax
Iqaluit
Lagos
Auckland
Santiago
Lima
Quito
Kigali
Bogota
London
Dublin
Berlin
Paris
Kyiv
Madrid
Asuncion
Buenos Aires
Sao Paulo
Fortaleza
Brussels
Jerusalem
Vienna
Malé
Dakar
Ulaanbaatar
Kingston
Flying Fish Cove
West Island
Nuku'alofa
Palikir
Saipan
Saint-Pierre
Maputo
Warsaw
Hagatna
San Juan
Charlotte Amalie
Antarctica
Port-au-Prince
Nassau
Lusaka
Fort-de-France
Marigot
Castries
Kingston
Basse-Terre
Saint George
Roseau
Gustavia
Bridgetown
Saint-Denis
Mayotte
Tbilisi
Santo Domingo
Port of Spain
Panama City
Tegucigalpa
Rome
Amsterdam
Chisinau
Copenhagen
Tallinn
Oslo
Tirana
Skopje
Valletta
Belgrade
Riga
Athens
Cayenne
San Salvador
San Jose
Guatemala City
Vilnius
Budapest
Sofia
Ljubljana
Bratislava
Honiara
Suva
Lilongwe
Porto-Novo
Helsinki
Lu